In [ ]:

import os
import pandas as pd
# import train_model
# import generate_synthetic
# import train_classifier
# import test_imgs
import numpy as np
import torch
from datetime import datetime
import os
import logging
from datetime import datetime
from random import seed


import split_dataset as split
import train_model

In [19]:

path_home = '/home/pathorad3090/Documents/Hadar/SyntheticEvaluation'
path_models = path_home + "/models"
path_raw_data = path_home + "/data/mnist_images"
path_cnn_classifier = path_home + '/cnn_classifier'

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [ ]:

# Set parameters for which train_size, gen_size, synthetic/real ratio, GAN train cutoff
# Set initial dir for experiments

# Set parameters as lists of integer values
seed = 42
train_sizes = [50, 100, 400, 800, 1000, 5000, 10000, 25000, 50000]  # List of different training sizes
gen_sizes = [50, 100, 400, 800, 1000, 5000, 10000, 25000, 50000]  # List of different generation sizes
synthetic_real_ratio = 0.5  # Ratio of synthetic to real data
gan_train_cutoff = 5000  # Number of GAN training iterations before switching
train_ratio = 0.8
kimg = 500
# Configure logging
timestamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
log_filename = (f"log_{timestamp}_seed{seed}_trainSizes{'_'.join(map(str, train_sizes))}_"
                f"genSizes{'_'.join(map(str, gen_sizes))}_kimg{kimg}.log")

logging.basicConfig(    filename=log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
# Example log messages
logging.info('Logging setup complete.')
logging.info('Parameters used: seed=%d, train_sizes=%s, gen_sizes=%s, kimg=%d', seed, train_sizes, gen_sizes, kimg)

# Log the configured parameters for verification
logger.info('log_filename: '+ log_filename)
logger.info("Training Configuration:")
logger.info(f"Train Sizes: {train_sizes}")
logger.info(f"Generation Sizes: {gen_sizes}")
# logger.info(f"Synthetic/Real Ratio: {synthetic_real_ratio}")
# logger.info(f"StyleGAN2-ADA Training Cutoff: {gan_train_cutoff}")
# logger.info(f"Train Ratio: {train_ratio}")
logger.info(f"Train Cutoff: {kimg}")
logger.info("-" * 40)  # Separator for clarity


In [ ]:
# Use preprocessing.ipynb to create a proper dataset
# Distribute files to relevant subfolders + create JSON


In [16]:
# Define paths and parameters
full_json = path_home + '/data/dataset_full.json'
train_json = full_json

logger.info(f"Loading data from {full_json} for splitting into train/test subsets.")

# Split into train/test subsets
train_df, test_df = split.split_train_test(train_json, train_ratio, seed=seed)

train_json = f"{path_raw_data}/train_data.json"
test_json = f"{path_raw_data}/test_data.json"

logger.info(f"Saving training data to {train_json}.")
split.save_json_dataset(train_df, train_json)

logger.info(f"Saving testing data to {test_json}.")
split.save_json_dataset(test_df, test_json)

logger.info("Printing class distribution for training and testing datasets.")
split.print_class_distribution(train_df, "Train")
split.print_class_distribution(test_df, "Test")

logger.info("Data splitting and saving completed successfully.")


2024-11-08 11:40:41,210 - INFO - Loading data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/dataset_full.json for splitting into train/test subsets.
2024-11-08 11:40:41,211 - INFO - Splitting data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/dataset_full.json with train ratio: 0.8
Splitting data: 100%|██████████| 10/10 [00:00<00:00, 104.74class/s]
2024-11-08 11:40:41,342 - INFO - Split completed: 47995 training samples and 12005 testing samples.
2024-11-08 11:40:41,346 - INFO - Saving training data to /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json.
2024-11-08 11:40:41,347 - INFO - Saving data to /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json
2024-11-08 11:40:41,421 - INFO - Data saved successfully: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json
2024-11-08 11:40:41,423 - INFO - Saving testing data to /home/pathorad3090/Docum

In [ ]:
# !export MKL_SERVICE_FORCE_INTEL=1

In [32]:
# train MLP/CNN classifier, test for benchmark using test_imgs.py
test_json = test_json
test_size = 10000


logger.info(f"Creating directories for test files: {path_cnn_classifier} json with size: {test_size}")
# _, _, path_model_images = split.open_folders(model_name, path_cnn_classifier)
path_classifier_data = path_cnn_classifier +"/data"
os.makedirs(path_cnn_classifier, exist_ok=True)
os.makedirs(path_classifier_data, exist_ok=True)


logger.info(f"Generating subset data from {test_json} with size {test_size} and seed {seed}")
subset_df = split.subset_data(train_json, test_size, seed)
# print(len(subset_df))
# subset_df


logger.info(f"Subset DataFrame created. Path raw data: {path_raw_data}, Path classifier data: {path_classifier_data}")
split.copy_images_to_model_and_dataset(subset_df, path_raw_data, path_classifier_data)

# logger.info(f"Saving subset data to {path_subset_json}")
# split.save_json_dataset(subset_df, path_subset_json)

# logger.info(f"Distributing files into label directories at {path_model_images}")
# split.distribute_files_to_label_dirs(path_model_images)

# logger.info("Generating labels JSON for the subset.")
# split.generate_labels_json(path_model_images, path_model_images, "dataset.json")


2024-11-08 11:46:40,936 - INFO - Creating directories for test files: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/cnn_classifier json with size: 10000
2024-11-08 11:46:40,937 - INFO - Generating subset data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/test_data.json with size 10000 and seed 42
2024-11-08 11:46:40,937 - INFO - Creating subset from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json with size: 10000
Subsetting classes: 100%|██████████| 10/10 [00:00<00:00, 2298.75class/s]
2024-11-08 11:46:40,988 - INFO - Subset creation completed with 10000 total samples.
2024-11-08 11:46:40,990 - INFO - Subset DataFrame created. Path raw data: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images, Path classifier data: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/cnn_classifier/data
2024-11-08 11:46:40,990 - INFO - Copying images from /home/pathorad3090/Documents/Hadar/SyntheticE

In [ ]:
# train_json = path_raw_data + '/train_data.json'
path_curr_run = os.path.join(path_models, 'run_' + datetime.now().strftime('%Y%m%d_%H-%M'))
# train_sizes = [40000]

for train_size in train_sizes:
    # Create a name for the subset and the model
    model_name = f"model_{train_size}_{datetime.now().strftime('%Y%m%d_%H-%M')}"
    path_model = os.path.join(path_curr_run, model_name)
    path_experiments = os.path.join(path_model, 'experiments')
    path_dataset = os.path.join(path_model, 'dataset')
    path_subset_json = f"{path_model}/dataset_subset_size_{train_size}_seed_{seed}.json"
    
    logger.info(f"Creating directories for model: {model_name}")
    _, _, path_model_images = split.open_folders(model_name, path_model)
    
    logger.info(f"Generating subset data from {train_json} with size {train_size} and seed {seed}")
    subset_df = split.subset_data(train_json, train_size, seed)
    
    logger.info(f"Subset DataFrame created. Path raw data: {path_raw_data}, Path model images: {path_model_images}")
    split.copy_images_to_model_and_dataset(subset_df, path_raw_data, path_model_images)
    
    logger.info(f"Saving subset data to {path_subset_json}")
    split.save_json_dataset(subset_df, path_subset_json)

    logger.info(f"Distributing files into label directories at {path_model_images}")
    split.distribute_files_to_label_dirs(path_model_images)
    
    logger.info("Generating labels JSON for the subset.")
    split.generate_labels_json(path_model_images, path_model_images, "dataset.json")
    
    logger.info(f"Creating dataset for {model_name}...")
    train_model.create_dataset(path_home, path_model_images, path_dataset)

    logger.info(f"Training model {model_name}...")
    train_model.run_stylegan_training(path_home, path_experiments, path_dataset, snap=10, kimg=kimg)
    
    logger.info(f"Cleaning up model directories for {model_name}")
    # split.delete_images_and_dataset_dirs(path_model)

        
        # train the classifier
        # split the training step and the gen steps


In [ ]:
path_curr_run = os.path.join(path_models, 'run_' + datetime.now().strftime('%Y%m%d_%H-%M'))
path_curr_run = '/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models'

for entry in os.scandir(path_curr_run):
    if entry.is_dir() and 'model' in str(entry.name):
        print(entry.name)
        path_model = entry
        path_latest_pkl_file = split.get_latest_pkl_file(path_model)
        if path_latest_pkl_file:
            logger.info(f"Most recent .pkl file: {path_latest_pkl_file}")
            for gen_size in gen_sizes:
                gen_size = gen_size // 10
                path_generations = os.path.join(path_model, 'generations_'+ str(gen_size * 10))
                logger.info(f"Generating synthetic images for {entry.name} with generation size {gen_size * 10}...")
                train_model.generate_stylegan_images(path_home, path_latest_pkl_file, path_generations, f"0-{gen_size}")
        else:
            logger.warning("No .pkl file found for generation.")

In [ ]:
path_curr_run = os.path.join(path_models, 'run_' + datetime.now().strftime('%Y%m%d_%H-%M'))
path_curr_run = '/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models'

for entry in os.scandir(path_curr_run):
    if entry.is_dir() and 'model' in str(entry.name):
        print(entry.name)
        


In [ ]:
# show results - relevant metric: avg/class accuracy, f1, precision, recall, AUC-ROC...
# graph/tabular
logging.shutdown()

